In [3]:
import os

import pandas as pd
import numpy  as np

from lib import pdb
from lib import nar

In [2]:
rres = ''
qres = ''

saveto  = None
saveres = None

rmsdmin = 0.
rmsdmax = 1e10

sizemin = 0.
sizemax = 1e10

rmsdsizemin = 0.
rmsdsizemax = 1e10

matchrange = 3.

rformat = 'PDB'
qformat = 'PDB'

saveformat = 'PDB'

threads = 1

repr_struct_comp = (
    nar.five_atom_repr,     # for primary alignment
    nar.five_atom_repr,     # to calculate centers of mass
    nar.three_atom_repr,    # for secondary alignment
    nar.three_atom_repr     # to calculate the RMSD
)

In [7]:
parser = pdb.Parser()

r = 'data/1xjr.cif'
q = 'data/1xjr.pdb'

rformat = 'blabla'
qformat = 'blablabla'

rname, rext = r.split(os.sep)[-1].split('.')
qname, qext = q.split(os.sep)[-1].split('.')

rext = rext.upper()
qext = qext.upper()

if rext in ['PDB', 'CIF']:
    rformat = rext

if qext in ['PDB', 'CIF']:
    qformat = qext

rstruct = parser(r, rformat, rname)
qstruct = parser(q, qformat, qname)

display(rstruct.tab.head(5), qstruct.tab.head(5))

,group_PDB,id,type_symbol,label_atom_id,label_alt_id,label_comp_id,label_asym_id,label_entity_id,label_seq_id,pdbx_PDB_ins_code,...,Cartn_y,Cartn_z,occupancy,B_iso_or_equiv,pdbx_formal_charge,auth_seq_id,auth_comp_id,auth_asym_id,auth_atom_id,pdbx_PDB_model_num
0,HETATM,1,P,PG,.,GTP,A,1,1,?,...,36.983,39.043,1.0,101.62,?,1,GTP,A,PG,1
1,HETATM,2,O,O1G,.,GTP,A,1,1,?,...,38.469,38.944,1.0,101.65,?,1,GTP,A,O1G,1
2,HETATM,3,O,O2G,.,GTP,A,1,1,?,...,36.367,37.772,1.0,102.06,?,1,GTP,A,O2G,1
3,HETATM,4,O,O3G,.,GTP,A,1,1,?,...,36.397,40.238,1.0,102.53,?,1,GTP,A,O3G,1
4,HETATM,5,O,O3B,.,GTP,A,1,1,?,...,36.691,39.242,1.0,100.71,?,1,GTP,A,O3B,1


,group_PDB,id,auth_atom_id,label_alt_id,auth_comp_id,auth_asym_id,auth_seq_id,pdbx_PDB_ins_code,Cartn_x,Cartn_y,Cartn_z,occupancy,B_iso_or_equiv,type_symbol,pdbx_formal_charge,pdbx_PDB_model_num
0,HETATM,1,PG,NaN,GTP,A,1,?,68.682,36.983,39.043,1.0,101.62,P,NaN,1
1,HETATM,2,O1G,NaN,GTP,A,1,?,68.400,38.469,38.944,1.0,101.65,O,NaN,1
2,HETATM,3,O2G,NaN,GTP,A,1,?,68.127,36.367,37.772,1.0,102.06,O,NaN,1
3,HETATM,4,O3G,NaN,GTP,A,1,?,67.953,36.397,40.238,1.0,102.53,O,NaN,1
4,HETATM,5,O3B,NaN,GTP,A,1,?,70.282,36.691,39.242,1.0,100.71,O,NaN,1


In [17]:
rres = '/A:A_10_20'
qres = '#1:_12A'

rsstruct = rstruct.get_sub_struct(rres)
qsstruct = qstruct.get_sub_struct(qres)

rsstruct.drop_duplicates_alt_id(keep='last')
qsstruct.drop_duplicates_alt_id(keep='last')

In [18]:
# pre proc repr_struct_comp
prep = []
repr_struct = {}
for repr_comp in repr_struct_comp:
    if repr_comp in prep:
        for k in repr_comp:
            if k in repr_struct:
                repr_struct[k].append(repr_comp[k])
            else:
                repr_struct[k] = [repr_comp[k]]
    else:
        for k in repr_comp.keys():
            repr_comp[k] = [pd.Index(v.split()) for v in repr_comp[k]]
            
            if k in repr_struct:
                repr_struct[k].append(repr_comp[k])
            else:
                repr_struct[k] = [repr_comp[k]]
        
        prep.append(repr_comp)
#

In [19]:
rdata = rsstruct.artem_desc(repr_struct)
qdata = qsstruct.artem_desc(repr_struct)